# Quantum Phase Estimation

This notebook present the method `qat.lang.algorithms.phase_estimation` that automatically builds phase estimation routines from an operator's implementation.

The method takes as input a unitary operator (usually a QRoutine object) and a number of bits of precision, and returns a QRoutine.

It is still up to the user to embed this routine in a proper quantum program and to prepare the data register so that it contains
the appropriate target quantum state (typically an eigenvector of the input operator).

We will use it to estimate the angle of a $Z$ rotation applied to a qubit prepare in state $|1\rangle$ !

In [ ]:
from re import X
import numpy as np
from qat.lang.AQASM import *
from qat.lang.algorithms import phase_estimation

phase_to_guess = np.pi / 4
print('The phase to guess is:', phase_to_guess)

operator = PH(phase_to_guess)
state_preparation = X
nbits = 3

prog = Program()
data = prog.qalloc(1)
# We declare the phase as a QInt, for conversion purposes.
phase = prog.qalloc(nbits, QInt)
# Preparing the desired eigenstate
state_preparation(data)
# applying the phase_estimation
phase_estimation(operator, nbits)(data, phase)
job = prog.to_circ().to_job(qubits=[phase], nbshots=1)
job.circuit.display()

What should we expect from this job?

The `phase` register should contain an approximation over 4 bits of the angle of the `PH` gate (divided by $2\pi$).
This entails that if we measure the `phase` register and obtain an integer value $b$, our guess for the phase is given by:
$$ \frac{2\pi b}{2^k} $$
where $k$ is the number of bits of precision.

Let us simulate this job and compute this value !

In [ ]:
from qat.qpus import get_default_qpu
qpu = get_default_qpu()


result = qpu.submit(job)
for sample in result:
    # Since we declared the phase register as a QInt, we get the bitstring to integer conversion for free using the .value attribute
    print("Our guess:", 2 * np.pi * sample.state.value[0] / (1 << nbits))
